# Notebook to test functions while the communication is not working

Libraries:
- pycryptodome : pip install pycryptodome
- cryptography : pip install cryptography
- pmlb: pip install pmlb
- pandas : pip install pandas

In [1]:
!pip uninstall pycrypto -y
!pip3 install pycryptodome pmlb

/bin/bash: pip: command not found
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from io import StringIO
from pmlb import fetch_data

from Crypto.Cipher import AES
from Crypto.Util import Counter
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

In [3]:
pd.read_csv('infringement_dataset.csv').head(3)

,loan_id,infringed,contract_type,gender,has_own_car,has_own_realty,num_children,annual_income,credit_amount,credit_annuity,...,SK_ID_CURR,avg_days_decision,past_avg_amount_annuity,past_avg_amt_application,past_avg_amt_credit,past_loans_approved,past_loans_refused,past_loans_canceled,past_loans_unused,past_loans_total
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,100002.0,606.0,9251.775,179055.0,179055.0,1.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,100003.0,1305.0,56553.990,435436.5,484191.0,3.0,0.0,0.0,0.0,3.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,100004.0,815.0,5357.250,24282.0,20106.0,1.0,0.0,0.0,0.0,1.0


#### Generate Private Key

In [4]:
def get_private_key():
    with open('/dev/urandom', 'rb') as f:
        return f.read(AES.key_size[0])

### AES-CTR

##### Encrypt

In [5]:
def encrypt_csv_ctr(file_name, civ, counter_size=128):
    key = get_private_key()
    
    counter = Counter.new(counter_size, initial_value=civ)
    
    aes = AES.new(key, mode=AES.MODE_CTR, counter=counter)

    with open(file_name, 'rb') as file:
        csv_data = file.read()
        encrypted_csv = aes.encrypt(csv_data)
    return encrypted_csv, key

ctr_iv = 10
ctr_ct, ctr_key = encrypt_csv_ctr('infringement_dataset.csv', ctr_iv)

#### Decrypt

In [6]:
def decrypt_csv_ctr(ct, key, counter_iv, counter_size=128):
    counter = Counter.new(counter_size, initial_value=counter_iv)    
    aes = AES.new(key=key, mode=AES.MODE_CTR, counter=counter)
    return aes.decrypt(ct)

In [7]:
pd.read_csv(StringIO(decrypt_csv_ctr(ctr_ct, ctr_key, ctr_iv).decode('utf-8'))).head(3)

,loan_id,infringed,contract_type,gender,has_own_car,has_own_realty,num_children,annual_income,credit_amount,credit_annuity,...,SK_ID_CURR,avg_days_decision,past_avg_amount_annuity,past_avg_amt_application,past_avg_amt_credit,past_loans_approved,past_loans_refused,past_loans_canceled,past_loans_unused,past_loans_total
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,100002.0,606.0,9251.775,179055.0,179055.0,1.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,100003.0,1305.0,56553.990,435436.5,484191.0,3.0,0.0,0.0,0.0,3.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,100004.0,815.0,5357.250,24282.0,20106.0,1.0,0.0,0.0,0.0,1.0
